In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import os
import struct
import numpy as np
from utils import *
import collections


In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
from glow import thops
from glow import modules
from glow import models
from glow.config import JsonConfig
import cv2

In [ ]:
# negative
path_total = '/home/hyshuai/workspace/dataset/images_background'

dataset_negative = []
for path_p in os.listdir(path_total):
        path_p_join = os.path.join(path_total,path_p)
        for path_c in os.listdir(path_p_join):
            path_real = os.path.join(path_p_join,path_c)
            class_name = path_real
            # image_paths = get_image_paths(path_real)
            images = get_images_frompath(path_real)
            print(class_name," : ",len(images))
            dataset_negative.append(ImageObject(class_name,images))

In [ ]:
# positive

path_positive_total = '/home/hyshuai/workspace/dataset/images_evaluation'
class_name_list = []
for path_p in os.listdir(path_positive_total):
    path_p_join = os.path.join(path_positive_total,path_p)
    for path_c in os.listdir(path_p_join):
        path_real = os.path.join(path_p_join,path_c)
        class_name = path_real
        class_name_list.append(str(class_name))

# class_name = np.array(class_name)
# print(class_name_list)
np.random.seed(66) # 切换类试下
shufindex = np.random.permutation(len(class_name_list))
# print(shufindex)
class_positive = np.array(class_name_list)[shufindex][:20] #取5个类做测试 5-way
print(class_positive)

dataset_train_positive =[]
dataset_test_positive = []
for class_path in class_positive :
    images = get_images_frompath_random(class_path) # 固定随机抽样是否有问题
    print(class_path, " : ", len(images))
    dataset_train_positive.append(ImageObject(class_path, images[:1])) # 1-shot
    dataset_test_positive.append(ImageObject(class_path, images[1:]))



print(len(dataset_train_positive[2].imgs))
print(len(dataset_test_positive[2].imgs))






    

In [ ]:
# label:

class_total_num = len(dataset_negative) + len(dataset_train_positive)
print("class_total = ",class_total_num)
np.random.seed(66) 
shufindex = np.random.permutation(class_total_num)
for index, value in enumerate(dataset_negative):
    value.label = shufindex[index]
    print(value.name,  value.label)

for index, value in enumerate(dataset_train_positive):
    value.label = shufindex[index+len(dataset_negative)]
    dataset_test_positive[index].label = shufindex[index+len(dataset_negative)]
    print(value.name, value.label)

In [ ]:
# test_class_index = [122,51,631,828,20]
test_class_index = [data.label for data in dataset_train_positive]
print(test_class_index)

In [ ]:
# print(dataset_test_positive[0].name)
# np.random.shuffle(dataset_test_positive)
# print(dataset_test_positive[0].name)
# imgs_show(np.squeeze( dataset_test_positive[0].imgs),1,5) 
# imgs_show(np.squeeze(dataset_negative[1].imgs),4,5)
class_total_num

In [ ]:
# print(dataset_negative[0].imgs[2].shape)
# # print(im[15])
# # print(im.shape)
# print(dataset_negative[0].imgs[0].shape)
x = [data.imgs for data in dataset_train_positive] 
print(x[0].shape)
img_show(np.squeeze(x[0]))
img_show(np.squeeze(x[1]))
img_show(np.squeeze(x[2]))
img_show(np.squeeze(x[3]))
img_show(np.squeeze(x[4]))

In [ ]:

def get_next_perfect(dataset_train_positive,dataset_negative,p_size,n_size):
    """
    final batch_size = p_size + n_size * 2
    every dataset_negative should have same imgs leng
    """
    #shuffle
    np.random.shuffle(dataset_train_positive)
    np.random.shuffle(dataset_negative)
    for data in dataset_negative:
        np.random.shuffle(data.imgs)
    for data in dataset_train_positive:
        np.random.shuffle(data.imgs)
    
    
    batch_negative = n_size * 2
#     print('batch_negative ',batch_negative)
    len_n_class = len(dataset_negative)
#     print('len_n_class ',len_n_class)
    len_p_class = len(dataset_train_positive)
#     print('len_p_class ',len_p_class)
    len_n_imgs = len(dataset_negative[0].imgs)
#     print('len_n_imgs ',len_n_imgs)
    len_p_imgs = len(dataset_train_positive[0].imgs)
    
    negative_number = 0
    
    
#     shuffindex = np.random.permutation(len_n)
# #     print(shuffindex)
#     np.random.shuffle(dataset_train_positive)
#     np.random.shuffle(dataset_negative)
    
    for data in dataset_negative:
        negative_number += len(data.imgs)
#         np.random.shuffle(data.imgs)
#     print('negative_number ',negative_number)
    batch_total = negative_number  // batch_negative
#     print('batch_total ',batch_total)
    i_n_class = 0
    i_n_img = 0
    i_p_class = 0
    i_p_img = 0
    
    
    
        
    
    for b in range(batch_total):
        train_x =[]
        train_y = []
        create_negative_imgs = 0
        while create_negative_imgs <= (batch_total * batch_negative) and create_negative_imgs < batch_negative:
            
            # while #如果一次循环没够，需要再次循环
            if i_n_class >= len_n_class:
                    i_n_class = 0
                    i_n_img +=1
                    if (i_n_img+1) *2 >len_n_imgs:
                        i_n_img = 0
#                     print('if i_n_class >= len_n_class i_n_class ',i_n_class)
#                     print('if i_n_class >= len_n_class i_n_img ',i_n_img)
            
            for i in range(i_n_class,len_n_class):
                    
#                 print('for i in range(i_n_class i_n_class ',i_n_class)
#                 print('for i in range(i_n_class i_n_img ',i_n_img)
                train_x += dataset_negative[i_n_class].imgs[i_n_img * 2 : (i_n_img+1) *2]
                train_y += [dataset_negative[i_n_class].label] * 2
                
                i_n_class+=1
                if i_n_class >= len_n_class:
                    i_n_class = 0
                    i_n_img +=1
                    if (i_n_img+1) *2 > len_n_imgs:
                        i_n_img = 0
#                     print('if i_n_class >= len_n_class i_n_class ',i_n_class)
#                     print('if i_n_class >= len_n_class i_n_img ',i_n_img)
            
                create_negative_imgs +=2
                if create_negative_imgs >= batch_negative:
#                     print("create_negative_imgs ok ",create_negative_imgs)
                    break
                    
                
        create_positive_imgs = 0    
        while create_positive_imgs < p_size:
            for i in range(i_p_class,len_p_class):
#                 print('for i in range(i_p_class     ',i_p_class)
#                 print('for i in range(i_p_img  ',i_p_img)
                
                train_x += dataset_train_positive[i_p_class].imgs[i_p_img:(i_p_img +1)]
                train_y += [dataset_train_positive[i_p_class].label]
                
                i_p_class+=1
                if i_p_class >= len_p_class:
                    i_p_class = 0
                    i_p_img +=1
                    if (i_p_img +1) > len_p_imgs:
                        i_p_img = 0
#                     print('if i_p_class >= len_p_class i_p_class ',i_p_class)
#                     print('if i_p_class >= len_p_class i_p_img ',i_p_img)
                
                create_positive_imgs+=1
                if create_positive_imgs >= p_size:
#                     print("create_positive_imgs ok ",create_positive_imgs)
                    break
        
        shuffindex_out = np.random.permutation(len(train_y))
#         print(train_x[0].shape)
        train_x = np.stack(train_x,axis = 0)
#         print('after concat : ',train_x[0].shape)
        train_x = train_x[shuffindex_out]
        train_y = np.array(train_y)[shuffindex_out]
        
        yield train_x,train_y  

In [ ]:
dataaaaaaa = get_next_perfect(dataset_train_positive,dataset_negative,4,126)
for x ,y in dataaaaaaa:
    print(y)
    print('x o shape ' ,x.shape,y.shape)
    print("one batch is over")


In [ ]:
def getnext_test(dataset_test_positive,test_class_index):
    
    for data in dataset_test_positive:
        x = np.stack(data.imgs,axis =0)
        y = np.array([test_class_index.index(data.label)] * len(data.imgs))
        yield x,y

In [ ]:
dataaaaaaa = getnext_test(dataset_test_positive,test_class_index)
for x ,y in dataaaaaaa:
    print(y)
    print(x.shape,y.shape)

In [ ]:
def imgs_show_norm(imgs,row,col):
    fig,ax = plt.subplots(nrows=row,ncols=col,sharex=True,sharey=True)
    ax = ax.flatten()
    for i in range(row*col):
        img = imgs[i]
        min = np.min(img)
#         print('min = ',min)
        img = np.subtract(img,min) # 0->
        max = np.max(img)
#         print('max =',max)
        img = np.divide(img,max)
        
        ax[i].imshow(img,cmap='Greys', interpolation='nearest')
    
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    plt.tight_layout()
    plt.show()

In [ ]:
def img_show_norm(img):
    min_ = np.min(img)
#     print('min = ',min_)
    img = np.subtract(img,min_) # 0->
    max_ = np.amax(a = img,keepdims=False)
#     print('max =',max_)
    img = np.divide(img,max_)  # 0->1
#     print(img)
    plt.figure(figsize=(1,1))
    plt.imshow(img,cmap='Greys',interpolation='nearest')

In [ ]:
def img_show(img):
    plt.figure(figsize=(1,1))
    plt.imshow(img,cmap='Greys',interpolation='nearest')
    

In [ ]:
def imgs_show(imgs,row,col):
    fig,ax = plt.subplots(nrows=row,ncols=col,sharex=True,sharey=True)
    ax = ax.flatten()
    for i in range(row*col):
        img = imgs[i]
        
        ax[i].imshow(img,cmap='Greys', interpolation='nearest')
    
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    plt.tight_layout()
    plt.show()
    
    

In [ ]:
nn_index = [-1,1]
def compute_index_logical_equal(inputs,indexs):
    logic_result = (inputs==indexs[0])
    for value in indexs:
        logic_result = np.logical_or(inputs == value,logic_result)
    return logic_result

def compute_index_logical_not_equal(inputs,indexs):
    logic_result = (inputs==indexs[0])
    for value in indexs:
        logic_result = np.logical_or(inputs == value,logic_result)
    return np.logical_not(logic_result)

In [ ]:
class AssociateGlowGenerated(nn.Module):
    def __init__(self, test_class_index, is_mean, K, class_number):
        super(AssociateGlowGenerated, self).__init__()
        self.glow = models.Glow(JsonConfig("hparams/omni_all_bg.json"), test_class_index=test_class_index , is_mean=is_mean, K=K, y_classes=class_number)
        self.class_number = class_number
        self.eval_index = test_class_index
        self.criterion = nn.MSELoss()
        self.ones_tensor = torch.ones((1, 32, 32)).float().cuda()

        self.glow_generate = models.Glow(JsonConfig("hparams/omni_all_bg.json"), test_class_index=test_class_index,
                                is_mean=is_mean, K=K, y_classes=class_number)

    def restore_glow_params(self,state_dict):
        self.glow.load_state_dict(state_dict)

    def resoter_glow_g_params(self,state_dict):
        self.glow_generate.load_state_dict(state_dict)


    def forward(self, inputs=None, labels=None, y_onehot=None, eps_std=None):

        x_shape = inputs.size()
        y_shape = labels.size()
        assert x_shape[0] == y_shape[0]
        #         print(labels)
        y_onehot = torch.FloatTensor(y_shape[0], self.class_number).cuda()  # if use gpu ,
        y_onehot.zero_()
        y_onehot.scatter_(1, labels.view(-1, 1), 1)
        filter_g_img = None
        #         print(y_onehot[0])

        
        #  generate image
        if self.training:
            with torch.no_grad():
                # decode input
                z, det = self.glow_generate(x=inputs, y_onehot=y_onehot, reverse=False)

                if not isinstance(labels, torch.Tensor):
                    labels = torch.from_numpy(labels)


                # z algorithm
                positive_index = compute_index_logical_equal(labels, self.eval_index)
                negitive_index = compute_index_logical_not_equal(labels, self.eval_index)
                #             print("positive_index:", positive_index)
                #             print("negitive_index:", negitive_index)

                z_negative = z[negitive_index]
                assert len(z_negative) % 2 == 0
                y_negative = labels[negitive_index]
                #             print('y_negative = ', y_negative)
                x_positive = z[positive_index]
                y_positive = labels[positive_index]
                #             print('y_positive = ', y_positive)

                sorted_y, indices_y = torch.sort(y_negative)
                z_negavite_sort = z_negative[indices_y]
                #             print('sorted_y = ', sorted_y)
                #             print('indices_y = ', indices_y)
                #             print('labels[indices_y] = ', labels[indices_y])
                #             print('y_negative[indices_y] = ', y_negative[indices_y])

                # sample
                z_size = z.size()
                batch_g_z_size = len(y_negative) // 2
                #             print('batch_g_size = ', batch_g_z_size)

                g_z = torch.zeros([batch_g_z_size, z_size[1], z_size[2], z_size[3]]).cuda()  # require_grad = False
                label_g_c = torch.zeros(batch_g_z_size).long().cuda()
                #             print('self.g_z = ', self.g_z.size())
                #             print('self.label_g_c = ', self.label_g_c.size())

                i = 0

                while i < batch_g_z_size:
                    for index_p in range(len(x_positive)):
                        g_z[i] = x_positive[index_p]

                        p = 0.6  # np.random.uniform(0.45, 0.5)
                        q = np.random.randint(0, 2)
                        scale = nn_index[q] * p

                        #                     start_index = np.random.randint(0,32)
                        #                     # start_index = 0
                        #                     end_index = start_index + 32
                        slic_index = np.random.permutation(64)[:20]
                        #                     print('scale = {},slic_index ={}'.format(scale,slic_index))
                        #                     print('y_positive[index_p] = {},y_negative[2 * i] ={}, y_negative[2 * i + 1] = {}'.
                        #                           format(y_positive[index_p],y_negative[indices_y][2 * i],y_negative[indices_y][2 * i + 1]))
                        #                     g_z[i][start_index:end_index] =  g_z[i][start_index:end_index] + \
                        #                                                                  scale * ( z_negavite_sort[2 * i][start_index:end_index] -
                        #                                                                                  z_negavite_sort[2 * i + 1][start_index:end_index])

                        g_z[i][slic_index] = g_z[i][slic_index] + scale * (
                                    z_negavite_sort[2 * i][slic_index] -
                                    z_negavite_sort[2 * i + 1][slic_index])

                        label_g_c[i] = y_positive[index_p]
                        i += 1
                        if i >= batch_g_z_size:
                            break

                #             print('label_g_c = ', label_g_c)
                # shuff_g_index = np.random.permutation(batch_g_z_size)
                # sg_z = .g_z[shuff_g_index]
                # .label_g_c = .label_g_c[shuff_g_index]

                # generate img

                y_onehot_g = torch.FloatTensor(batch_g_z_size, self.class_number).cuda()  # if use gpu ,
                y_onehot_g.zero_()
                y_onehot_g.scatter_(1, label_g_c.view(-1, 1), 1)

                g_img = self.glow_generate(z=g_z, y_onehot=y_onehot_g, eps_std=eps_std, reverse=True)
                # print('g_img requires_grad = ', g_img.requires_grad)
                g_img = torch.clamp(g_img, -1, 1)
                g_img = g_img.detach()
                #                 print('.g_img requires_grad = ', .g_img.requires_grad)

                filter_index_list = []
                # filter_g_label_list = []
                # filter_g_oneshot_list = []

                for i in range(len(g_img)):
                    g_corr_label = (labels == label_g_c[i])
                    #                     print('labels[g_corr_label ={}, label_g_c[i] = {}'.format(.label_g_c[i],labels[g_corr_label]))
                    diff_g = self.criterion(g_img[i], inputs[g_corr_label][0]) / self.criterion(
                        inputs[g_corr_label][0], self.ones_tensor)
                    #                     print('diff_g = ',diff_g)

                    if diff_g < 1:
                        filter_index_list.append(i)

                print('filter_index_ len  = ', len(filter_index_list))
                filter_g_img = g_img[filter_index_list]
                #                 print('filter_g_img shape = ', .filter_g_img.size())
                filter_g_label = label_g_c[filter_index_list]
                filter_g_oneshot = y_onehot_g[filter_index_list]

                # shuff_g_index = np.random.permutation(len(.filter_g_label))
                # .filter_g_img = .filter_g_img[shuff_g_index]
                # .filter_g_label = .filter_g_label[shuff_g_index]
                # .filter_g_oneshot = .filter_g_oneshot[shuff_g_index]

                if len(filter_g_img) != 0:
                    inputs = torch.cat((inputs, filter_g_img), 0)
                    print('input_real shape = ', inputs.size())
                    labels = torch.cat((labels, filter_g_label), 0)
                    print('label_real shape = ', labels.size())
                    y_onehot = torch.cat((y_onehot,filter_g_oneshot),0)
                    print('onehot_real shape = ', y_onehot.size())

                    shuff_g_index = np.random.permutation(len(inputs))
                    inputs = inputs[shuff_g_index]
                    labels = labels[shuff_g_index]
                    y_onehot= y_onehot[shuff_g_index]

                   













        # classify
        z, det = self.glow(x=inputs, y_onehot=y_onehot, reverse=False)
        y_logits = self.glow.class_flow(z)
        return z, det, y_logits ,labels,filter_g_img

In [ ]:
class AssociateGlow(nn.Module):
    def __init__(self, test_class_index, is_mean, K, class_number):
        super(AssociateGlow, self).__init__()
        self.glow = models.Glow(JsonConfig("hparams/omni_all_bg.json"), test_class_index=test_class_index \
                                , is_mean=is_mean, K=K, y_classes=class_number)
        self.class_number = class_number
        self.eval_index = test_class_index
        self.criterion = nn.MSELoss()
        self.ones_tensor = torch.ones((1,32,32)).float().cuda()

    def forward(self, inputs=None, labels=None, y_onehot=None, eps_std=None):

        x_shape = inputs.size()
        y_shape = labels.size()
        assert x_shape[0] == y_shape[0]

#         print(labels)
        y_onehot = torch.FloatTensor(y_shape[0], self.class_number).cuda()  # if use gpu ,
        y_onehot.zero_()
        y_onehot.scatter_(1, labels.view(-1, 1), 1)
#         print(y_onehot[0])

        # decode z
        z, det = self.glow(x=inputs, y_onehot=y_onehot, reverse=False)

        if self.training:
            if not isinstance(labels, torch.Tensor):
                labels = torch.from_numpy(labels)

            # z algo
            positive_index = compute_index_logical_equal(labels, self.eval_index)
            negitive_index = compute_index_logical_not_equal(labels, self.eval_index)
#             print("positive_index:", positive_index)
#             print("negitive_index:", negitive_index)

            z_negative = z[negitive_index]
            assert len(z_negative) % 2 == 0
            y_negative = labels[negitive_index]
#             print('y_negative = ', y_negative)
            x_positive = z[positive_index]
            y_positive = labels[positive_index]
#             print('y_positive = ', y_positive)

            sorted_y, indices_y = torch.sort(y_negative)
            z_negavite_sort = z_negative[indices_y]
#             print('sorted_y = ', sorted_y)
#             print('indices_y = ', indices_y)
#             print('labels[indices_y] = ', labels[indices_y])
#             print('y_negative[indices_y] = ', y_negative[indices_y])


            # sample
            z_size = z.size()
            batch_g_z_size = len(y_negative) // 2
#             print('batch_g_size = ', batch_g_z_size)

            self.g_z = torch.zeros( [batch_g_z_size, z_size[1], z_size[2], z_size[3]]).cuda()  # require_grad = False
            self.label_g_c = torch.zeros(batch_g_z_size).long().cuda()
#             print('self.g_z = ', self.g_z.size())
#             print('self.label_g_c = ', self.label_g_c.size())

            i = 0
            

            while i < batch_g_z_size:
                for index_p in range(len(x_positive)):
                    self.g_z[i] = x_positive[index_p]
                    
                    p = 0.6 # np.random.uniform(0.45, 0.5)
                    q = np.random.randint(0, 2)
                    scale = nn_index[q] * p

#                     start_index = np.random.randint(0,32)
#                     # start_index = 0
#                     end_index = start_index + 32
                    slic_index = np.random.permutation(64)[:20]
#                     print('scale = {},slic_index ={}'.format(scale,slic_index))
#                     print('y_positive[index_p] = {},y_negative[2 * i] ={}, y_negative[2 * i + 1] = {}'.
#                           format(y_positive[index_p],y_negative[indices_y][2 * i],y_negative[indices_y][2 * i + 1]))
#                     self.g_z[i][start_index:end_index] =  self.g_z[i][start_index:end_index] + \
#                                                                  scale * ( z_negavite_sort[2 * i][start_index:end_index] -
#                                                                                  z_negavite_sort[2 * i + 1][start_index:end_index])
                        
                    self.g_z[i][slic_index] =  self.g_z[i][slic_index] + scale * ( z_negavite_sort[2 * i][slic_index] - 
                                                                                  z_negavite_sort[2 * i + 1][slic_index])
                        
                    self.label_g_c[i] = y_positive[index_p]
                    i += 1
                    if i >= batch_g_z_size:
                        break

#             print('self.label_g_c = ', self.label_g_c)
            # shuff_g_index = np.random.permutation(batch_g_z_size)
            # self.g_z = self.g_z[shuff_g_index]
            # self.label_g_c = self.label_g_c[shuff_g_index]

            # generate img

            self.y_onehot_g = torch.FloatTensor(batch_g_z_size, self.class_number).cuda()  # if use gpu ,
            self.y_onehot_g.zero_()
            self.y_onehot_g.scatter_(1, self.label_g_c.view(-1, 1), 1)

            with torch.no_grad():  # generate net grad ------------------- 需要更新的 因为x_c 一直在更新
                self.g_img = self.glow(z=self.g_z, y_onehot=self.y_onehot_g, eps_std=eps_std, reverse=True)
                # print('self.g_img requires_grad = ', self.g_img.requires_grad)
                self.g_img = torch.clamp(self.g_img, -1, 1)
                self.g_img = self.g_img.detach()
#                 print('self.g_img requires_grad = ', self.g_img.requires_grad)

                filter_index_list = []
                # filter_g_label_list = []
                # filter_g_oneshot_list = []

                for i in range(len(self.g_img)):
                    g_corr_label = (labels == self.label_g_c[i])
#                     print('labels[g_corr_label ={}, label_g_c[i] = {}'.format(self.label_g_c[i],labels[g_corr_label]))
                    diff_g = self.criterion (self.g_img[i] , inputs[g_corr_label][0]) / self.criterion (inputs[g_corr_label][0],self.ones_tensor)
#                     print('diff_g = ',diff_g)

                    if diff_g < 1:
                        filter_index_list.append(i)

                print('filter_index_ len  = ', len(filter_index_list))
                self.filter_g_img = self.g_img[filter_index_list]
#                 print('filter_g_img shape = ', self.filter_g_img.size())
                self.filter_g_label = self.label_g_c[filter_index_list]
                self.filter_g_oneshot =self.y_onehot_g[filter_index_list]
        
                shuff_g_index = np.random.permutation(len(self.filter_g_label))
                self.filter_g_img = self.filter_g_img[shuff_g_index]
                self.filter_g_label =self.filter_g_label[shuff_g_index]
                self.filter_g_oneshot = self.filter_g_oneshot[shuff_g_index]

#                 print('filter_g_label shape = ', self.filter_g_label.size())
#                 print('filter_g_label  = ', self.filter_g_label)
#                 print('filter_g_oneshot shape = ', self.filter_g_oneshot.size())

                # g_ = torch.norm(torch.norm(torch.norm(self.g_img, 2, 1), 2, 1), 2, 1) ** 2


        if self.training and len(self.filter_g_img) != 0 :
            # classify
            z_g, det_g = self.glow(x=self.filter_g_img, y_onehot=self.filter_g_oneshot, reverse=False)
#             print('det_g requires_grad = ', z_g.requires_grad)

        y_logits = self.glow.class_flow(z)
            
        if self.training and len(self.filter_g_img) != 0 :
            y_logits_g = self.glow.class_flow(z_g)
#             print('y_logits_g size  = ', y_logits_g.size())
            return det, det_g, y_logits, y_logits_g, self.filter_g_label
        if self.training:
            return det,None,y_logits,None,self.filter_g_label
        return det,  y_logits

In [ ]:
# nb_digits = class_total_num
# net_auto = AssociateGlow(test_class_index,is_mean=True,K= 8,class_number=class_total_num)
net_auto = AssociateGlowGenerated(test_class_index,is_mean=True,K= 8,class_number=class_total_num)

In [ ]:
#     print(k)
pt1 = torch.load('model_glow_ominst_all_bg_8k_20_way_778.pt')
pt2 = torch.load('model_glow_ominst_all_bg_8k_20_way_778.pt')

In [ ]:
# for k ,v in pt_asso.items():
#     print(k)
#     print(v)
# for k ,v in pm.items():


net_auto.restore_glow_params(pt1)
net_auto.resoter_glow_g_params(pt2)

In [ ]:
print(pt1.has_key)
print(pt2.has_key)

In [ ]:

pt_old = torch.load('model_glow_ominst_all_bg_8k_20_way_778.pt')
# pm = net_auto.state_dict()
pt_asso = collections.OrderedDict()
for k ,v in pt_old.items():
#     print(k)
#     print('glow.'+k)
#     print(type(k))
    pt_asso['glow.'+k] = v
#     print(v)
    

In [ ]:

a = net_auto.load_state_dict(pt_asso)
# for param in net_auto.parameters():
#     print(param.name)
a = net_auto.parameters().next()
print(a)
print(pt_old['flow.layers.1.actnorm.bias'])

In [ ]:

# net_auto.load_state_dict(pt_asso)
# net_auto.load_state_dict(torch.load('model_assoiatet_glow_ominst_all_bg_200.pt'))
# net_auto.load_state_dict(torch.load('model_assoiatet_glow_ominst_all_bg_77.pt'))
# net_auto.load_state_dict(torch.load('model_glow_ominst_no_split2d_4k_4316.pt'))
# net_auto.load_state_dict(torch.load('model_glow_ominst_no_split2d_8k_9000.pt'))
use_cuda = torch.cuda.is_available()
# torch.manual_seed(666)
device = torch.device("cuda" if use_cuda else "cpu")
net_auto.to(device)
net_auto.float()


criterion = nn.MSELoss()
optimizer = optim.Adam(net_auto.parameters(),lr=0.0001,weight_decay=1e-5) # decoder don't weight_decay


In [ ]:
#X_train_nor
# batch_size = 512
net_auto.train()
for epoch in range(200):
    #train:
    
    datas =  get_next_perfect(dataset_train_positive,dataset_negative,16,200)
#     datas = getnext(dataset_train_positive,dataset_negative)
    
    trainloss_g = 0
    trainloss_c = 0
    
    for x,y in datas:
        inputs ,lables = torch.from_numpy(x).float().to(device),\
            torch.from_numpy(y).long().to(device)
#         print(torch.sum(lables)) # = 1167
        
#         y_onehot = torch.FloatTensor(batch_size, nb_digits).to(device)
#         y_onehot.zero_()
#         y_onehot.scatter_(1, lables.view(-1,1), 1)
#         print('y_onehot:', y_onehot)

        optimizer.zero_grad()
#         det,det_g, y_logits,y_logit_g,label_g_c = net_auto(inputs=inputs, labels = lables ,y_onehot=None,eps_std=None)
        z, det, y_logits ,labels_g,filter_g_img = net_auto(inputs=inputs, labels = lables ,y_onehot=None,eps_std=None)
#         print(z.size())
#         print(det)
        
        loss_g1 = models.Glow.loss_generative(det)
#         loss_g2 = models.Glow.loss_generative(det_g)
    
        loss_c1 = models.Glow.loss_class(y_logits,labels_g)
#         loss_c2 = models.Glow.loss_class(y_logit_g,label_g_c)
        
#         print('loss_g1 = ',loss_g1.item())
#         print('loss_g2 = ',loss_g2.item())
#         print('loss_c1 =  ',loss_c1.item())
#         print('loss_c2 =  ',loss_c2.item())
        print('%d  loss_g1: %.4f,loss_c1: %.4f ' % (epoch , loss_g1.item(),loss_c1.item()))
#         print('label_g_c =  ',label_g_c[:25])
#         if epoch > 20:
#         if filter_g_img is not None and len(filter_g_img) > 25 and epoch > 20:
#             imgs_show(np.squeeze(filter_g_img),5,5)
        
        loss = loss_g1  + loss_c1 * 0.2 
#         loss = loss_g1 + + loss_c1 * 0.2
        loss.backward()
        torch.nn.utils.clip_grad_value_(net_auto.parameters(), 5)
        torch.nn.utils.clip_grad_norm_(net_auto.parameters(), 100)
        
        
#         trainloss_g += (loss_g1.item()+ loss_g2.item())
#         trainloss_c += (loss_c1.item() + loss_c2.item())
#         print("net_auto.conv_d1 grad:",net_auto.conv_d1.weight.grad)
        
#         print("after net_auto.conv_1 grad:",net_auto.conv1.weight.grad)
        optimizer.step()
    
        
        
    print('%d  gloss: %.3f, class loss:%.3f' % (epoch ,  trainloss_g,trainloss_c))
    
    if epoch % 10 == 0 and epoch >0:
        torch.save(net_auto.state_dict(),'model_assoiatet_glow_ominst_all_bg_ge_nograd_'+str(epoch)+'.pt')
    


In [ ]:
# net_auto.load_state_dict(torch.load('model_assoiatet_glow_ominst_all_bg_ge_nograd_10.pt'))
use_cuda = torch.cuda.is_available()
torch.manual_seed(666)
device = torch.device("cuda" if use_cuda else "cpu")
net_auto.to(device)
net_auto.float()
net_auto.eval()

test_loss = 0
correct = 0
test_datas = getnext_test(dataset_test_positive,test_class_index)
for x ,y in test_datas:
        inputs, labels = Variable(torch.from_numpy(x).float()).cuda(), Variable(torch.from_numpy(y).long()).cuda()
        z, det, y_logits ,labels_g,filter_g_img = net_auto(inputs=inputs, labels = labels ,y_onehot=None,eps_std=None)
        
#         _,classify = net_auto(inputs)
        test_loss += models.Glow.loss_class(y_logits,labels)  # sum up batch loss
#         print(test_loss)
        pred = y_logits.max(1,keepdim=True)[1]
#         print(pred)
        
        correct += pred.eq(labels.view_as(pred)).sum().item()
#         print(correct)
print(correct)
print(correct * 1.0 / (19 * 20))
print(test_loss)

In [ ]:
# net_auto.load_state_dict(torch.load('model_glow_ominst2000.pt'))


# use_cuda = torch.cuda.is_available()
# torch.manual_seed(666)
# device = torch.device("cuda" if use_cuda else "cpu")
# net_auto.to(device)
# net_auto.float()
# net_auto.load_state_dict(torch.load('model_glow_ominst_no_split2d_no_z_mean_no_add_normal1223.pt'))
# torch.cuda.empty_cache()
# net_auto.load_state_dict(torch.load('model_assoiatet_glow_ominst_all_bg_900.pt'))
net_auto.train()
torch.save(net_auto.state_dict(),'model_assoiatet_glow_ominst_all_bg_ge_nograd_50.pt')

In [ ]:
posi = np.concatenate(( np.array(dataset_train_positive[3].imgs[1:2])  ,  np.array(dataset_negative[112].imgs[1:3])   ))
labl =  np.array([dataset_train_positive[3].label, dataset_negative[12].label,dataset_negative[112].label])

inputs ,lables = torch.from_numpy(posi).float().to(device),\
            torch.from_numpy(labl).long().to(device)
        
y_onehot = torch.FloatTensor(3, nb_digits).to(device)
y_onehot.zero_()
y_onehot.scatter_(1, lables.view(-1,1), 1)
# print('y_onehot:', y_onehot)
z, det, y_logits = net_auto(x=inputs, y_onehot=y_onehot)


print('z.size() ' ,z.size())
print('y_logits.size() ' ,y_logits.size())
print(y_logits[1][1])

In [ ]:
obj = z[0]
# print(obj[0])
obj = z[0] 

print(torch.sum(torch.abs(z[2]),dim =(1,2)))
print(torch.sum(torch.abs(z[1]),dim =(1,2)))

print(torch.sum(torch.abs(z[0]),dim =(1,2)))
obj[:][:] = z[0][:] + 0.5 * (z[1][:] - z[2][:])

print(obj.size())
obj = obj.view(-1,*obj.size())
print(obj.size())
obj_oneshot = y_onehot[0].view(-1,class_total_num)
print(obj_oneshot.size())
# print(obj_oneshot)
# print(obj[0][0])

In [ ]:
# z = z.detach().cpu().numpy()
# print(z.size())
# print(z[0][0][0])
x_ = net_auto(z = obj,y_onehot = obj_oneshot,eps_std = None ,reverse=True)

x_ = torch.clamp(x_,-1,1)
print(x_.size())
print(torch.sum(torch.abs(x_ - inputs[0])))
print(x_[0][0][0])
print(inputs[0][0][0])

In [ ]:
img_show(np.squeeze(x_.data))
img_show(np.squeeze(inputs.data[0]))
imgs_show(np.squeeze(inputs.data),1,3)